In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import gmaps.datasets

# Import API key
from config1 import gmap_key

# Configure gmaps
gmaps.configure(api_key=gmap_key)
!jupyter nbextension enable --py gmaps
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,4,East London,ZA,55.80,overcast clouds,-33.0153,27.9116,Tu Casa
1,6,Georgetown,MY,85.93,few clouds,5.4112,100.3354,Cititel Penang
2,7,Rikitea,PF,75.15,broken clouds,-23.1203,-134.9692,Pension Maro'i
3,9,Mataura,NZ,49.48,overcast clouds,-46.1927,168.8643,Ellie's Villa
4,11,Ngorongoro,TZ,49.01,overcast clouds,-3.2397,35.4875,Simba Special Campsite B


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
map_fig = gmaps.figure(center=(-30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
map_fig.add_layer(marker_layer)
# 4b. Display the figure
map_fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df["City"] == "Plettenberg Bay") & (vacation_df["Country"] == "ZA")]
vacation_end = vacation_df.loc[(vacation_df["City"] == "Plettenberg Bay") & (vacation_df["Country"] == "ZA")]
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Bredasdorp") & (vacation_df["Country"] == "ZA")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Hermanus") & (vacation_df["Country"] == "ZA")] 
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "Saldanha") & (vacation_df["Country"] == "ZA")] 

In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = [tuple(coord) for coord in vacation_start[["Lat", "Lng"]].to_numpy()][0]
end = [tuple(coord) for coord in vacation_end[["Lat", "Lng"]].to_numpy()][0]
stop1 = [tuple(coord) for coord in vacation_stop1[["Lat", "Lng"]].to_numpy()][0]
stop2 = [tuple(coord) for coord in vacation_stop2[["Lat", "Lng"]].to_numpy()][0]
stop3 = [tuple(coord) for coord in vacation_stop3[["Lat", "Lng"]].to_numpy()][0]

In [7]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
direction_fig = gmaps.figure(center=(-30.0, 31.0), zoom_level=5)
vacation_it = gmaps.directions_layer(start, end, waypoints = [stop1, stop2, stop3], travel_mode = 'DRIVING')
direction_fig.add_layer(vacation_it)
direction_fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# 8. Create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
del itinerary_df["City_ID"]
itinerary_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Plettenberg Bay,ZA,55.51,overcast clouds,-34.0527,23.3716,Thanda Vista B&B
1,Bredasdorp,ZA,47.52,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
2,Hermanus,ZA,56.89,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
3,Saldanha,ZA,50.05,light rain,-33.0117,17.9442,Blue Bay Lodge


In [9]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [10]:
# 11a. Add a marker layer for each city to the map.
itinerary_fig = gmaps.figure(center=(-30, 31.0), zoom_level=5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
itinerary_fig.add_layer(marker_layer)
# 11b. Display the figure
itinerary_fig

Figure(layout=FigureLayout(height='420px'))